## Importando variáveis de ambiente
Esse notebook prevê a existência de 2 variáveis de ambiente no arquivo .env desse projeto:
- DATA_FOLDER
- DATASET_FOLDER
- TRAINED_MODELS_FOLDER

In [2]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env")

DATA_FOLDER = os.getenv("DATA_FOLDER")
DATASET_FOLDER = os.getenv("DATASET_FOLDER")
TRAINED_MODELS_FOLDER = os.getenv("TRAINED_MODELS_FOLDER")

### Reajuste opcional nos nomes das pastas do dataset
DESCOMENTE E EXECUTE ESSE CÓDIGO CASO O NOME DAS PASTAS DE CLASSE NO DATASET CONTENHAM ESPAÇOS EM BRANCO.

In [ ]:
# def rename_subfolders(base_path):
#     # Percorre todas as subpastas dentro da pasta base
#     for root, dirs, files in os.walk(base_path, topdown=False):
#         for d in dirs:
#             old_path = os.path.join(root, d)
#             new_name = d.replace(" ", "_")
#             new_path = os.path.join(root, new_name)

#             # Só renomeia se houver mudança
#             if old_path != new_path:
#                 print(f"Renomeando: {old_path} -> {new_path}")
#                 os.rename(old_path, new_path)

# rename_subfolders(DATASET_FOLDER)

## Bibliotecas

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from PIL import Image
import numpy as np

import sqlite3
import faiss
from tqdm import tqdm

## Configurações

In [4]:
dataset_path = DATASET_FOLDER  # pasta principal com subpastas das classes
sqlite_db_path = os.path.join(DATA_FOLDER, "metadata.db")
faiss_dir = os.path.join(DATA_FOLDER, "faiss_indexes")
os.makedirs(faiss_dir, exist_ok=True)

model_path = os.path.join(TRAINED_MODELS_FOLDER, "best_resnet50.pth")  # caminho do modelo treinado

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 30

## Transform

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

## Carregar a ResNet50 pré-treinada

In [6]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

model.fc = torch.nn.Linear(model.fc.in_features, num_classes) # seta as classes de peixes

model.load_state_dict(torch.load(model_path, map_location=device))

model.eval()
model.to(device)

feature_extractor = nn.Sequential(*list(model.children())[:-1]) # seta para geração de embeddings na última camada fully-connected (2048D)
feature_extractor.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


## Criação do banco de dados SQLite para os metadados das imagens

In [18]:
dataset = datasets.ImageFolder(dataset_path, transform=transform)
class_to_idx = dataset.class_to_idx

conn = sqlite3.connect(sqlite_db_path)
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS class (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE,
    index_path TEXT
);
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS image (
    id INTEGER PRIMARY KEY,
    id_class INTEGER,
    image_path TEXT,
    FOREIGN KEY (id_class) REFERENCES class(id)
);
""")

conn.commit()

## Inserção das classes

In [19]:
print("Indexando classes segundo ImageFolder...")

class_indices = {}            # nome_da_classe -> class_id (hasmap do nome da classe para o seu id)
faiss_indices = {}            # class_id -> FaissIndex (hashmap de class_id para índice, por classe)

for class_name, class_id in class_to_idx.items():
    # Inserir classe no SQLite usando class_id do ImageFolder
    cur.execute(
    f"""
        INSERT INTO class (id, name)
        VALUES ({class_id}, '{class_name}')
    """)
    conn.commit()

    class_indices[class_name] = class_id

    # criar índice FAISS para essa classe
    index = faiss.IndexFlatL2(2048)
    index = faiss.IndexIDMap(index)
    faiss_indices[class_id] = index

Indexando classes segundo ImageFolder...


## Inserção das imagens

In [ ]:
print("Extraindo embeddings e populando FAISS...")
for class_name, class_id in class_to_idx.items():
    class_path = os.path.join(dataset_path, class_name)
    
    if not os.path.isdir(class_path):
        continue

    class_id = class_indices[class_name]

    for img_name in os.listdir(class_path):
        if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        img_path = os.path.join(class_path, img_name)

        # Extrai vetor de características
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            feat = feature_extractor(img_tensor)
            feat = feat.view(feat.size(0), -1)
            # normalização dos valores (ideal para cosine/IP)
            feat = torch.nn.functional.normalize(feat, p=2, dim=1)
            # converter para CPU + numpy float32 (necessário, pois FAISS para versão do python utilizada não suporta GPU)
            feat_np = feat.cpu().numpy().astype("float32")

        # Registra no SQLite a 
        cur.execute(
        f"""
            INSERT INTO image (id_class, image_path)
            VALUES ({class_id}, '{img_path}')
        """)
        conn.commit()

        img_id = cur.lastrowid  # último id salvo no banco

        # Insere vetor no índice FAISS dessa classe
        faiss_indices[class_id].add_with_ids(
            feat_np,
            np.array([img_id], dtype="int64")
        )


print("\nSalvando índices FAISS...")

for class_id, index in faiss_indices.items():
    index_path = f"{faiss_dir}/{class_id}.index"

    # salvar índice
    faiss.write_index(index, index_path)

    # atualizar SQLite com o index_path
    cur.execute(
    f"""
        UPDATE class
        SET index_path = '{index_path}'
        WHERE id = {class_id}
    """)

conn.commit()

Extraindo embeddings e populando FAISS...

Salvando índices FAISS...


## Criando índice

In [23]:
print("\nOtimizando banco de dados...")

cur.execute("CREATE INDEX IF NOT EXISTS idx_images_class ON image(id_class);")
cur.execute("CREATE INDEX IF NOT EXISTS idx_images_path ON image(image_path);")
cur.execute("CREATE INDEX IF NOT EXISTS idx_classes_name ON class(name);")
conn.commit()

conn.close()

print("\nProcesso concluído!")
print(f"Banco SQLite salvo em: {sqlite_db_path}")
print(f"Índices FAISS salvos em: {faiss_dir}/")


Otimizando banco de dados...

Processo concluído!
Banco SQLite salvo em: /home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/metadata.db
Índices FAISS salvos em: /home/tatmiki/Documentos/Prog_Projects/cnn_peixes/data/faiss_indexes/
